In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey

from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
import functions_pebble_accretion as peb
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch

from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 
import sim_loader as sim_load



In [ ]:
# disc parameters
params_dict = {'St_const': None, 
               'iceline_radius': None,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'epsilon_el': 1e-2,
                'epsilon_heat':0.5,
                'v_frag': (1 * u.m/u.s).to(u.au/u.Myr).value,
                'M_dot_gas_star': "Hartmann_2016",
                'iceline_v_frag_change': True,
                }
params = code_gas.Params(**params_dict)
a_p0 = np.geomspace(1,1, num =1 )
t0 = 0.1
t_fin = 5
mdot_star = M_dot_star(t0, params)
H_r0 = H_R(a_p0, mdot_star, params)
sigma_gas = sigma_gas_steady_state(a_p0, H_r0, mdot_star, params)
m0 = M0_pla_Mstar(a_p0, H_r0, sigma_gas, params)

a_p0 = np.array([a_p0])
m_0 = np.array([m0])
t0 = np.array([t0])
t_fin = 3 #Myr, end of sim
N_steps = 500 #number of steps of the sim 

sim_params_dict = {'N_step': N_steps,
                'm0': m_0,
                'a_p0': a_p0,
                't0': t0,
                't_fin': t_fin,
            }
sim_params = code_gas.SimulationParams(**sim_params_dict)
peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = peb.GasAccretion()

#output_folder = 'sims/gas_acc/test_speedup/'
output_folder = 'sims/gas_acc/test_speedup/new/speedup/'


cProfile.run('code_gas.simulate_euler(migration = True, filtering = True, peb_acc = peb_acc, gas_acc=gas_acc, params=params, sim_params=sim_params, output_folder=output_folder)', 'profile_results_new.prof')

In [ ]:
# Create a Stats object and sort the output by total time
stats = pstats.Stats('profile_results_new.prof')
stats.strip_dirs().sort_stats('cumulative').print_stats(200)


In [ ]:
fig, axs = plt.subplots(1,1, figsize=(6,4))
sim = sim_load.JSONtoSimRes("sims/gas_acc/test_speedup/simulation_Lambrechts_mixed_e_el_0.01_vfrag_1.0_planets_1_t0_0.1_N_steps500_Mstar_1.0_Z_0.01.json")
sim_new = sim_load.JSONtoSimRes("sims/gas_acc/test_speedup/new/speedup/simulation_Lambrechts_mixed_e_el_0.01_vfrag_1.0_planets_1_t0_0.1_N_steps500_Mstar_1.0_Z_0.01.json")
print(len(sim.position[0,:][0]), len(sim_new.position[0,:][0]))
t_test = -1
axs.scatter(sim.position[0,0][:t_test], sim.mass[0,0][:t_test], s=5)
axs.scatter(sim_new.position[0,0][:t_test], sim_new.mass[0,0][:t_test], s=5, marker='x')
axs.axvline(r_magnetic_cavity(M_dot_star(sim.time[0].value, params), params), color='grey', linestyle='--', label='Magnetic cavity radius at t0')
axs.axvline(r_magnetic_cavity(M_dot_star(sim.time[-1].value, params), params), color='black', linestyle='--', label='Magnetic cavity radius at t_fin')
axs.axvline(r_magnetic_cavity(M_dot_star(sim.time[t_test].value, params), params), color='red', linestyle='--', label='Magnetic cavity radius at t')
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_xlabel('Semi-major axis [au]')
axs.set_ylabel('Mass [Mearth]')
axs.set_title('Test speedup gas accretion')

In [ ]:
params = code_gas.Params(M_dot_gas_star="Hartmann_2016", H_r_model="irradiated")


def M0_pla_Mstar_new(position, H_r, sigma_gas, params):
    """Initial planetesimal mass scaling accordin to Eq. 14 in Liu et al. 2019b """
    rho_gas =  rho_0 (position, H_r, sigma_gas)
    if params.self_gravity:
        gamma=1
    else:
        gamma = (4*np.pi*G*rho_gas)/(omega_k(position, params)**2)
    C = 5e-5
    a = 0.5
    b = 0
    f = 400
    return 2e-3*(f/400)*(C/5e-5)*(gamma*np.pi)**(a+1)*(H_r/0.05)**(3+b)*(params.star_mass/(0.1*M_sun_M_E))*(1*u.M_earth).value
def H_R_old(position, t, params):
    """H/r possible modes, from Ida, Liu, Bitsch"""
    
    if params.H_r_model == "Lambrechts_mixed":
        return np.maximum(H_R_irr(position, params), H_R_visc_Lambrechts_old(position, t, params))

    if params.H_r_model == 'irradiated':
        return H_R_irr(position, params)

def H_R_visc_Lambrechts_old(position, t, params):
    """Viscous H/r from equation (A47) in Lambrechts notes"""
    return 0.019*(params.epsilon_el/1e-2)**(1/10)*(params.epsilon_heat/0.5)**(1/10)*(params.alpha/1e-2)**(-1/10)*(params.Z/0.01)**(1/10)*(params.a_gr/(0.1*mm_to_au))**(-1/10)*(params.rho_gr/g_cm3_to_M_E_au3)**(-1/10)*(position)**(1/20)*(M_dot_star(t, params)/(1e-8*M_sun_yr_to_M_E_Myr))**(1/5)*(params.star_mass/M_sun_M_E)**(-7/20)


def rho_0_old (position, t, sigma_gas, params):
    """Midplane gas density assuming isothermal disc"""
    return sigma_gas/(np.sqrt(2*np.pi)*H_R_old(position, t, params)*position)
def sigma_gas_steady_state_old(position, t, params):
    """Gas surface density for a steady state viscously evolving disc, equation (11) in Ida et al. 2016"""
    return M_dot_star(t, params)/(3*np.pi*params.alpha*H_R_old(position, t, params)**2*position**2*omega_k(position, params))


def M0_pla_Mstar_old(position, t, sigma_gas, params):
    """Initial planetesimal mass scaling accordin to Eq. 14 in Liu et al. 2019b """
    rho_gas =  rho_0_old (position, t, sigma_gas, params)
    if params.self_gravity:
        gamma=1
    else:
        gamma = (4*np.pi*G*rho_gas)/(omega_k(position, params)**2)
    C = 5e-5
    a = 0.5
    b = 0
    f = 400
    return 2e-3*(f/400)*(C/5e-5)*(gamma*np.pi)**(a+1)*(H_R_old(position, t, params)/0.05)**(3+b)*(params.star_mass/(0.1*M_sun_M_E))*(1*u.M_earth).value

a_p0 = np.geomspace(1e-2, 10, num =1000 )
t0 = 0.1
t_fin = 5
print("mdot star", M_dot_star(t0, params))
print("mdot star", M_dot_star_t(t0))
print(params.M_dot_gas_star)

mdot_star = M_dot_star(t0, params)
H_r0 = H_R(a_p0, mdot_star, params)
sigma_gas = sigma_gas_steady_state(a_p0, H_r0, mdot_star, params)
sigma_gas_old = sigma_gas_steady_state_old(a_p0, t0, params)

fig,axs = plt.subplots(1,1, figsize=(6,4))
axs.plot(a_p0, M0_pla_Mstar_new(a_p0, H_r0, sigma_gas, params), label='new function')
axs.plot(a_p0, M0_pla_Mstar_old(a_p0, t0, sigma_gas, params), label='old function', linestyle='dashed')
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_xlabel('Semi-major axis [au]')
axs.set_ylabel('Initial planetesimal mass [Mearth]')
axs.set_title('Comparison initial planetesimal mass functions')
axs.legend()

fig,axs = plt.subplots(1,1, figsize=(6,4))
axs.plot(a_p0, sigma_gas, label='new function')
axs.plot(a_p0, sigma_gas_old, label='old function', linestyle='dashed')
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_xlabel('Semi-major axis [au]')
axs.set_ylabel('Gas surface density [g/cm^2]')
axs.set_title('Comparison gas surface density functions')

fig,axs = plt.subplots(1,1, figsize=(6,4))
axs.plot(a_p0, H_R(a_p0, mdot_star, params), label='new function')
axs.plot(a_p0, H_R_old(a_p0, t0, params), label='old function', linestyle='dashed')
axs.set_xscale('log')
axs.set_yscale('log')   
axs.set_xlabel('Semi-major axis [au]')
axs.set_ylabel('H/r')
axs.set_title('Comparison H/r functions')
axs.legend()
dt = 10000*u.yr.to(u.Myr)
fig, axs = plt.subplots(1,1, figsize=(6,4))
axs.scatter(sim.mass[0,:], sim.dM_dt[0,:]*dt/sim.mass[0,:], s=5)
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_xlabel('Mass [Mearth]')
axs.set_ylabel('Growth rate [1/Myr]')
axs.set_title('Mass growth rate')

In [ ]:
fig,axs = plt.subplots(1,1, figsize=(6,4))
t = np.geomspace(0.1, 5, num =100)
mdot_star = M_dot_star(t, params)
axs.plot(t, r_magnetic_cavity(mdot_star, params))
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_xlabel('Time [Myr]')
axs.set_ylabel('Magnetic cavity radius [au]')
axs.set_title('Magnetic cavity radius vs time')